# Quickstart

Start with `Flower Datasets` as fast as possible by learning the essentials.

In [2]:
!python --version

Python 3.12.11


每次启动都要重新运行安装各种包，下面这行运行两遍才能成功

In [6]:
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib

In [7]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

Training on cpu
Flower 1.20.0 / PyTorch 2.8.0+cu126


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## Install Flower Datasets

In [8]:
! pip install -q "flwr-datasets[vision]"

If you want to use audio datasets install:

```bash
! pip install -q "flwr-datasets[audio]"
```

flag1-25-9-4

## Choose the dataset

To choose the dataset, go to Hugging Face [Datasets Hub](https://huggingface.co/datasets) and search for your dataset by name. You will pass that names to the `dataset` parameter of `FederatedDataset`. Note that the name is case-sensitive.

<div style="max-width:80%; margin-left: auto; margin-right: auto;">
  <img src="https://github.com/fycq22/flower/blob/main/datasets/docs/source/_static/tutorial-quickstart/choose-hf-dataset.png?raw=1" alt="Choose HF dataset."/>
</div>

Note that once the dataset is available on HuggingFace Hub it can be immediately used in `Flower Datasets` (no approval from Flower team is needed, no custom code needed).

Here is how it looks for `CIFAR10` dataset.

<div style="max-width:80%; margin-left: auto; margin-right: auto;">
  <img src="https://github.com/fycq22/flower/blob/main/datasets/docs/source/_static/tutorial-quickstart/copy-dataset-name.png?raw=1" alt="Choose HF dataset."/>
</div>

## Partition the dataset

To partition a dataset (in a basic scenario), you need to choose two things:
1) A dataset (identified by a name),
2) A partitioning scheme (by selecting one of the supported partitioning schemes, [see all of them here](https://flower.ai/docs/datasets/ref-api/flwr_datasets.partitioner.html), or creating a custom partitioning scheme).



**1) Dataset choice**

We will pass the name of the dataset to `FederatedDataset(dataset="some-name", other-parameters)`. In this example it will be: `FederatedDataset(dataset="uoft-cs/cifar10", other-parameters)`

**2) Partitioner choice**

We will partition the dataset in an IID manner using `IidPartitioner` ([link to the docs](https://flower.ai/docs/datasets/ref-api/flwr_datasets.partitioner.IidPartitioner.html#flwr_datasets.partitioner.IidPartitioner)).
Only the train split of the dataset will be processed. In general, we do `FederatedDataset(dataset="some-name", partitioners={"split-name": partitioning_scheme})`, which for this example looks like:

由于我们只有一个csv文件，因此需要自己划分测试集和训练集。此外我们还需要把Label从value修改为Classlabel。

In [11]:
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner

# 创建 FederatedDataset，只对 train 做分区
fds = FederatedDataset(
    dataset="fycq22/CICIDS-171819",
    partitioners={"train": IidPartitioner(num_partitions=10)},
)

# 加载第一个 partition (client 0 的训练集)
partition = fds.load_partition(0, "train")

# 加载完整的 train split（未分区）
full_train = fds.load_split("train")

# 从 train split 中划分出唯一的 centralized test
dataset = full_train.train_test_split(test_size=0.2, seed=42, stratify_by_column="Label")
centralized_test = dataset["test"]

print(f"Partition size (client 0 train): {len(partition)}")
print(f"Centralized test size: {len(centralized_test)}")

/usr/local/lib/python3.12/dist-packages/flwr_datasets/utils.py:109: UserWarning: The currently tested dataset are ['mnist', 'ylecun/mnist', 'cifar10', 'uoft-cs/cifar10', 'fashion_mnist', 'zalando-datasets/fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet', 'scikit-learn/adult-census-income', 'cifar100', 'uoft-cs/cifar100', 'svhn', 'ufldl-stanford/svhn', 'sentiment140', 'stanfordnlp/sentiment140', 'speech_commands', 'LIUM/tedlium', 'flwrlabs/femnist', 'flwrlabs/ucf101', 'flwrlabs/ambient-acoustic-context', 'jlh/uci-mushrooms', 'Mike0307/MNIST-M', 'flwrlabs/usps', 'scikit-learn/iris', 'flwrlabs/pacs', 'flwrlabs/cinic10', 'flwrlabs/caltech101', 'flwrlabs/office-home', 'flwrlabs/fed-isic2019']. Given: fycq22/CICIDS-171819.
  warnings.warn(


ValueError: Stratifying by column is only supported for ClassLabel column, and column Label is Value.

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Now we have 10 partitions created from the train split of the CIFAR10 dataset and the test split
for the centralized evaluation. Later we will convert the type of the dataset from Hugging Face's `Dataset` type to the format required by PyTorch/TensorFlow frameworks.

## Investigate the partition

### Features

Now we will determine the names of the features of your dataset (you can alternatively do that directly on the Hugging Face
website). The names can vary along different datasets e.g. "img" or "image", "label" or "labels". Additionally, if the label column is of [ClassLabel](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.ClassLabel) type, we will also see the names of labels.

In [ ]:
# Note this dataset has
partition.features

{'img': Image(mode=None, decode=True, id=None),
 'label': ClassLabel(names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], id=None)}

### Indexing

To see the first sample of the partition, we can index it like a Python list.

In [ ]:
partition[0]

{'img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
 'label': 1}

Then we can additionally choose the specific column.

In [ ]:
partition[0]["label"]

1

We can also use slicing (take a few samples). Let's take the first 3 samples of the first partition:

In [ ]:
partition[:3]

{'img': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>],
 'label': [1, 2, 6]}

We get a dictionary where the keys are the names of the columns and the values are list of the corresponding values of each row of the dataset. So to take the first 3 labels we can do:

In [ ]:
partition[:3]["label"]

[1, 2, 6]

Note that the indexing by column first is also possible but discouraged because the whole column will be loaded into the memory.

In [ ]:
partition["label"][:3]

[1, 2, 6]

You can also select a subset of the dataset and keep the same type (dataset.Dataset) instead of receiving a dictionary of values.

In [ ]:
partition.select([0, 1, 2])

Dataset({
    features: ['img', 'label'],
    num_rows: 3
})

And this dataset contains the same samples as we saw before.

In [ ]:
partition.select([0, 1, 2])[:]

{'img': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>],
 'label': [1, 2, 6]}

## Use with PyTorch/NumPy/TensorFlow

For more detailed instructions, go to:

* [how-to-use-with-pytorch](https://flower.ai/docs/datasets/how-to-use-with-pytorch.html)

* [how-to-use-with-numpy](https://flower.ai/docs/datasets/how-to-use-with-numpy.html)

* [how-to-use-with-tensorflow](https://flower.ai/docs/datasets/how-to-use-with-tensorflow.html)

### PyTorch

Transform the `Dataset` into the `DataLoader`, use the `PyTorch transforms` (`Compose` and all the others are possible).

In [ ]:
! pip install -q torch torchvision

In [ ]:
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

transforms = ToTensor()


def apply_transforms(batch):
    # For CIFAR-10 the "img" column contains the images we want to apply the transforms to
    batch["img"] = [transforms(img) for img in batch["img"]]
    return batch


partition_torch = partition.with_transform(apply_transforms)
dataloader = DataLoader(partition_torch, batch_size=64)

The `Dataloader` created this way does not return a `Tuple` when iterating over it but a `Dict` with the names of the columns as keys and features as values. Look below for an example.

In [ ]:
for batch in dataloader:
    print(f"Return type when iterating over a dataloader: {type(batch)}")
    print(batch["img"].shape)
    print(batch["label"].shape)
    break

Return type when iterating over dataloader: <class 'dict'>
torch.Size([64, 3, 32, 32])
torch.Size([64])


### NumPy

NumPy can be used as input to the TensorFlow and scikit-learn models. The transformation is very simple.

In [ ]:
partition_np = partition.with_format("numpy")
X_train, y_train = partition_np["img"], partition_np["label"]

### TensorFlow Dataset

Transformation to TensorFlow Dataset is a one-liner.

In [ ]:
! pip install -q tensorflow

In [ ]:
tf_dataset = partition.to_tf_dataset(
    columns="img", label_cols="label", batch_size=64, shuffle=True
)

## Final remarks

Congratulations, you now know the basics of Flower Datasets and are ready to perform basic dataset preparation for Federated Learning.

## Next

This is the first quickstart tutorial from the Flower Datasets series. See other tutorials:

* [Use Partitioners](https://flower.ai/docs/datasets/tutorial-use-partitioners.html)

* [Visualize Label Distribution](https://flower.ai/docs/datasets/tutorial-visualize-label-distribution.html)